<h1> Import CSV Vendor Exports - Combine and Create Master Product Lifecycle Dict </h1>

# Import Modules and Define Variables

In [ ]:
# data import and file manipulation
import os
import requests
import json
import csv
import xlrd

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

# Import and Parse Product Lifecycle Dicts

In [ ]:
df_wiki_release_info = pd.read_csv(f'{dictionary_dir}/product_lifecycle/wiki_versions.dict')

In [ ]:
def buildnum_from_version(version):
    try:
        result = (re.match(r'Version\s(\d+)',version)).group(1)
        for index, row in df_wiki_release_info.iterrows():
            try:
                if result == row['Version']:
                    # print(f"{result} == {row['Version']}")
                    # print(f"Build number == {row['Build number']}")
                    return row['Build number']
                    break
            except:
                pass
    except:
        pass

In [ ]:
def match_release(row):
    try:
        product_breakdown = {}
        result = re.match(r'^(Windows\s(XP|7|8[^\s]+|10|11|Server))([\s\d\w\.]+)?',row['Product'])
        if result:
            product_breakdown['productName'] = (result.group(1))
            product_breakdown['productVariation'] = result.group(3).lstrip()
            if re.match(r'^Windows\s(XP|7|8[^\s]+|10|11)',row['Product']):
                product_breakdown['shortProductName'] = product_breakdown['productName'] + " " + row['Edition']
            product_breakdown['fullProductName'] = row['Product']

            middle = row[['Edition','Release']].to_dict()

            try:
                product_breakdown['BuildNumber'] = buildnum_from_version(row['Release'])
            except Exception as e:
                print(e)

            product_breakdown.update(middle)

            last = row[['SupportPolicy','StartDate','MainstreamDate','ExtendedEndDate','RetirementDate','ReleaseStartDate','ReleaseEndDate','DocsUrl']].to_dict()
            product_breakdown.update(last)

            return product_breakdown
        else:
            pass
    except Exception as e:

        pass

In [ ]:
url = "https://app-omaha-prod.azurewebsites.net/api/PublishedListings/Export"
df_product_release_info = pd.read_excel(url,skiprows=6)
df_product_release_info.fillna('[NA]',inplace=True)

In [ ]:
windows_mask = df_product_release_info['Product'].str.contains('^Windows (XP|7|8|8.1|10|11|Server|Server\s?\d+?\s?\d+?)')

df_windows_release_info = df_product_release_info[windows_mask]

recombine_list = []

for index, row in df_windows_release_info.iterrows():
    try:
        if match_release(row):
            recombine_list.append(match_release(row))
    except:
        pass

df_combined = pd.DataFrame(recombine_list)

### Fix Time to_day

In [ ]:
for index, row in df_combined.iterrows():
    accumulator = set()
    for k,v in row.items():
        try:
            if type(v) == pd.Timestamp:
                accumulator.add(k)
            # else:
            #     try:
            #         dt.datetime.strptime("%Y-%m-%dT%H:%M:%S",v)
            #         print(v)
            #         if type(v) == pd.Timestamp:
            #             accumulator.add(k)
            #
            #     except:
            #         pass
        except:
            pass

date_cols = list(accumulator)
print(date_cols)

In [ ]:
def time_to_day(date_value):
    print(date_value)
    try:
        if type(date_value) == pd.Timestamp:
            print(f"\nbefore {date_value}")
            print(f"after {date_value.to_period('d')}","\n","*"*50)
            return date_value.to_period('d')
        else:
            return date_value
    except Exception as e:
        print(e)
        pass

In [ ]:
for e in date_cols:
    df_combined[e] = df_combined[e].apply(time_to_day)

In [ ]:
df_combined